In [ ]:
import torch 
import torch.nn as nn
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from keras.datasets import mnist
plt.rcParams['figure.figsize'] = [20, 10]

In [ ]:
# device = torch.device('cpu')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
class NeuralNet(nn.Module):
    def __init__(self, input_shape, output_shape, seed=43):
        super(NeuralNet, self).__init__()
        torch.manual_seed(seed)
        torch.cuda.manual_seed(seed)

        self.fc1 = nn.Linear(input_shape, 500)
        self.fc2 = nn.Linear(500, 50)
        self.fc3 = nn.Linear(50, output_shape)

        self.relu = nn.ReLU()

        self.do20 = nn.Dropout(p=0.2)

        self.report = dict()
        self.report['train_loss'] = []
        self.report['train_re'] = []
        self.report['test_loss'] = []
        self.report['test_re'] = [] 
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.do20(out)

        out = self.fc2(out)
        out = self.relu(out)

        out = self.fc3(out)
        return out

    def next_mini_batch(self, dataset, batch_size):
        N = dataset['count']
        indxs = np.arange(N)
        np.random.shuffle(indxs)

        for i in range(0, N-batch_size+1, batch_size):
            mini_batch = indxs[i:i+batch_size]
            yield zip(dataset['X'][mini_batch], dataset['y'][mini_batch])

    def test(self, criterion, testdata):
        mean_loss = 0.0

        with torch.no_grad():
            for x, y in zip(testdata['X'], testdata['y']):
                x, y = x.to(device), y.to(device)
                out = self(x)
                loss = criterion(out, y)
                mean_loss += loss.item()

        self.report['test_loss'].append(mean_loss/testdata['count'])

    def learn(self, criterion, epochs, traindata, testdata, 
            r_lambda=0, optimizer=None, lr_schedulers=None, 
            nesterov=False, eta=0.001, batch_size=32, verbos=1):

        if optimizer:
            optimizer = optimizer[0](self.parameters(), lr=eta,
                                     weight_decay=r_lambda, **optimizer[1])
        else:
            optimizer = torch.optim.SGD(self.parameters(), lr=eta,
                                        momentum=0.9, weight_decay=r_lambda,
                                        nesterov=nesterov)

        for i in tqdm(range(epochs), disable=(not verbos)):
            mean_loss = 0.0
            for mini_batch in self.next_mini_batch(traindata, batch_size):
                batch_loss = 0.0
                for x, y in mini_batch:
                    x, y = x.to(device), y.to(device)
                    out = self(x)
                    loss = criterion(out, y)
                    
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()
                    batch_loss += loss.item()
                mean_loss += batch_loss
            self.report['train_loss'].append(mean_loss/traindata['count'])

            self.test(criterion, testdata)

            if verbos == 1:
                print("train loss:{} test loss:{}" 
                      .format(self.report['train_loss'][-1], 
                              self.report['test_loss'][-1]))

    def weight_init(self, m):
        if isinstance(m, nn.Linear):
            nn.init.xavier_uniform_(m.weight, gain=nn.init.calculate_gain('relu'))
            nn.init.zeros_(m.bias)

    def reset(self):
        self.apply(self.weight_init)
        self.report['train_loss'] = []
        self.report['test_loss'] = []

In [ ]:
def make_dataset():
    (x_train, y_train), (x_test, y_test) = mnist.load_data()

    train_count = 60000
    test_count = 10000

    traindata = {'count': train_count,
                'X': torch.from_numpy((x_train/255).reshape(-1, 784).astype('float32')),
                'y': torch.from_numpy((y_train/9).reshape(-1, 1).astype('float32'))}
    testdata = {'count': test_count,
                'X': torch.from_numpy((x_test/255).reshape(-1, 784).astype('float32')),
                'y': torch.from_numpy((y_test/9).reshape(-1, 1).astype('float32'))}

    return traindata, testdata

In [ ]:
epochs = 10

traindata, testdata = make_dataset()

RMSprop = (torch.optim.RMSprop, {'alpha':0.99, 'eps':1e-08,
                                 'momentum':0, 'centered':False})
Adam = (torch.optim.Adam, {'betas':(0.9, 0.999), 'eps':1e-08,
                           'amsgrad':False})
Adadelta = (torch.optim.Adadelta, {'rho':0.9, 'eps':1e-06})
Adagrad = (torch.optim.Adagrad, {'lr_decay':0, 'initial_accumulator_value':0,
                                'eps':1e-10})

mynet = NeuralNet(784, 1).to(device)
mynet.reset()
mynet.learn(nn.SmoothL1Loss(), epochs, traindata, testdata,
            optimizer=RMSprop, batch_size=32, eta=0.001)
print(mynet.report['train_loss'][-1], mynet.report['test_loss'][-1])
plt.plot(np.arange(20, epochs), mynet.report['train_loss'][20:],
         label="train")
plt.plot(np.arange(20, epochs), mynet.report['test_loss'][20:],
         label="test")

plt.legend()
plt.show()



  0%|          | 0/10 [00:00<?, ?it/s]

 10%|█         | 1/10 [01:28<13:19, 88.82s/it]

train loss:0.010533410413328245 test loss:0.006495079702108803




 20%|██        | 2/10 [02:56<11:47, 88.48s/it]

train loss:0.006241323654660821 test loss:0.007345224026967262




 30%|███       | 3/10 [04:23<10:16, 88.11s/it]

...



 50%|█████     | 5/10 [07:17<07:17, 87.50s/it]

train loss:0.004784157222229015 test loss:0.0049643642709847075




 60%|██████    | 6/10 [08:44<05:49, 87.27s/it]

train loss:0.0045358804697160126 test loss:0.005262128732954901


KeyboardInterrupt: ignored

In [ ]:
from google.colab.patches import cv2_imshow
index = np.random.randint(0, 10000, size=(20))
for i in index:
    img = (testdata['X'][i].reshape(28,28,1) * 255).numpy().astype('uint8')
    cv2_imshow(img)
    with torch.no_grad():
        x, y = (testdata['X'][i]).to(device), (testdata['y'][i]).to(device)
        out = mynet(x)
        print(out*9)

tensor([7.0738], device='cuda:0')


tensor([9.3165], device='cuda:0')


tensor([1.7911], device='cuda:0')


tensor([6.1583], device='cuda:0')


tensor([1.5089], device='cuda:0')


...

tensor([6.1606], device='cuda:0')


tensor([1.7481], device='cuda:0')


tensor([0.7811], device='cuda:0')


tensor([7.4474], device='cuda:0')


tensor([3.9995], device='cuda:0')
